In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras as kr

from sklearn.metrics import euclidean_distances

2023-01-08 17:39:39.576550: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 17:39:39.901320: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-08 17:39:39.978641: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-08 17:39:39.978660: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

# Load the dataset

In [2]:
df = pd.read_csv("/home/vmh/vmhdocs/Research/Inria/Anl/MetaGenAutoencoder/Data/KO_metaG.norm.txt",sep="\t")
df = df.iloc[:,2-len(df.columns):]
print(df.shape)
df.head(5)

train_data = np.array(df)
print(train_data.shape)
inputlen = len(df.columns)
print(inputlen)

(180, 9024)
(180, 9024)
9024


# Load the model

In [3]:
autoencoder = kr.models.load_model("./autoencoder_metagendata.h5")
autoencoder.summary()

2023-01-08 17:39:43.127783: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-08 17:39:43.127999: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-08 17:39:43.128015: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vmhpc): /proc/driver/nvidia/version does not exist
2023-01-08 17:39:43.128500: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 9024)]            0         
                                                                 
 dense_6 (Dense)             (None, 1180)              10649500  
                                                                 
 batch_normalization_5 (Batc  (None, 1180)             4720      
 hNormalization)                                                 
                                                                 
 dense_7 (Dense)             (None, 590)               696790    
                                                                 
 batch_normalization_6 (Batc  (None, 590)              2360      
 hNormalization)                                                 
                                                                 
 dense_8 (Dense)             (None, 3)                 1773

# Fit with the dataset

In [4]:

checkpoint_filepath = './modelcheckpoint'
chkpt = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    monitor='loss',
    verbose=1)

tb = kr.callbacks.TensorBoard(log_dir="./model_tb",update_freq="epoch")
earlystop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=500)


autoencoder.fit(train_data,train_data,shuffle=True, epochs=50000, 
                            batch_size=50, verbose=2,callbacks=[chkpt,tb,earlystop])



Epoch 1/50000

Epoch 1: loss improved from inf to 0.21672, saving model to ./modelcheckpoint
INFO:tensorflow:Assets written to: ./modelcheckpoint/assets
4/4 - 2s - loss: 0.2167 - mean_squared_error: 0.2167 - 2s/epoch - 595ms/step
Epoch 2/50000

Epoch 2: loss improved from 0.21672 to 0.12552, saving model to ./modelcheckpoint
INFO:tensorflow:Assets written to: ./modelcheckpoint/assets
4/4 - 2s - loss: 0.1255 - mean_squared_error: 0.1255 - 2s/epoch - 423ms/step
Epoch 3/50000

Epoch 3: loss improved from 0.12552 to 0.09275, saving model to ./modelcheckpoint
INFO:tensorflow:Assets written to: ./modelcheckpoint/assets
4/4 - 2s - loss: 0.0928 - mean_squared_error: 0.0928 - 2s/epoch - 466ms/step
Epoch 4/50000

Epoch 4: loss improved from 0.09275 to 0.07808, saving model to ./modelcheckpoint
INFO:tensorflow:Assets written to: ./modelcheckpoint/assets
4/4 - 25s - loss: 0.0781 - mean_squared_error: 0.0781 - 25s/epoch - 6s/step
Epoch 5/50000

Epoch 5: loss improved from 0.07808 to 0.06716, saving